# Import Packages

In [ ]:
# add the directory containing modules to the path
import sys
sys.path.append('../modules')

In [ ]:
################################################################################
# NUMPY
# conda install numpy

import numpy as np

################################################################################
# SCIPY
# conda install scipy

# import scipy as sp

################################################################################
# MATPLOTLIB
# conda install matplotlib

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

################################################################################
# SEABORN
# conda install seaborn

import seaborn as sns

################################################################################
# PYLTTB - Time Series Downsampling Using Largest-Triangle-Three-Buckets
# pip install pylttb

# from pylttb import lttb

################################################################################
# NEO
# pip install neo>=0.7.1
# - AxoGraph support requires axographio to be installed: pip install axographio

# import neo

################################################################################
# QUANTITIES
# conda install quantities

import quantities as pq
pq.markup.config.use_unicode = True  # allow symbols like mu for micro in output
pq.mN = pq.UnitQuantity('millinewton', pq.N/1e3, symbol = 'mN');  # define millinewton

################################################################################
# ELEPHANT
# pip install git+https://github.com/NeuralEnsemble/elephant.git@master

import elephant

################################################################################
# PANDAS
# conda install pandas

# import pandas as pd

################################################################################
# STATSMODELS
# conda install statsmodels

import statsmodels.api as sm

################################################################################
# SPM1D - One-Dimensional Statistical Parametric Mapping
# pip install spm1d

# import spm1d

################################################################################
# EPHYVIEWER
# pip install git+https://github.com/jpgill86/ephyviewer.git@experimental
# - requires PyAV: conda install -c conda-forge av

# import ephyviewer

################################################################################
# ParseMetadata
# - requires ipywidgets: conda install ipywidgets
# - requires yaml:       conda install pyyaml

from ParseMetadata import LoadMetadata

################################################################################
# ImportData

from ImportData import LoadAndPrepareData

################################################################################
# NeoUtilities

# from NeoUtilities import CausalAlphaKernel

################################################################################
# EphyviewerConfigurator

# from EphyviewerConfigurator import EphyviewerConfigurator

################################################################################
# NeoToEphyviewerBridge

# from NeoToEphyviewerBridge import NeoSegmentToEphyviewerSources#, PlotExampleWithEphyviewer

# IPython Magics

In [ ]:
# make figures interactive and open in a separate window
# %matplotlib qt

# make figures interactive and inline
%matplotlib notebook

# make figures non-interactive and inline
# %matplotlib inline

# Data Parameters

In [ ]:
feeding_condition_parameters = [
    ('JG05 regular nori', 'IN VIVO / JG05 / 2018-03-05 / 001', ['I2',       'BN2', 'BN3'], [[ 550,  594]], ['Swallow (regular 5-cm nori strip)']),
    ('JG05 tape nori',    'IN VIVO / JG05 / 2018-03-05 / 001', ['I2',       'BN2', 'BN3'], [[ 745,  827]], ['Swallow (tape nori)']),
    ('JG07 regular nori', 'IN VIVO / JG07 / 2018-05-20 / 002', ['I2', 'RN', 'BN2', 'BN3'], [[1496, 1527]], ['Swallow (regular 5-cm nori strip)']),
    ('JG07 tape nori',    'IN VIVO / JG07 / 2018-05-20 / 002', ['I2', 'RN', 'BN2', 'BN3'], [[1036, 1093]], ['Swallow (tape nori)']),
    ('JG08 fresh food',   'IN VIVO / JG08 / 2018-06-25 / 001', ['I2', 'RN', 'BN2', 'BN3'], [[1170, 1590]], ['Swallow (fresh food)']),
    ('JG08 regular nori', 'IN VIVO / JG08 / 2018-06-21 / 001', ['I2', 'RN', 'BN2', 'BN3'], [[2462, 2532]], ['Swallow (regular 5-cm nori strip)']),
    ('JG08 tape nori',    'IN VIVO / JG08 / 2018-06-21 / 002', ['I2', 'RN', 'BN2', 'BN3'], [[ 134,  205]], ['Swallow (tape nori)']),
    ('JG08 tubing',       'IN VIVO / JG08 / 2018-06-25 / 001', ['I2', 'RN', 'BN2', 'BN3'], [[4026, 4146]], ['Swallow (tubing)', 'No movement (tubing)', 'Reposition (tubing)', 'Rejection (tubing)']),
    ('JG08 two-ply nori', 'IN VIVO / JG08 / 2018-06-25 / 001', ['I2', 'RN', 'BN2', 'BN3'], [[3256, 3493]], ['Swallow (two-ply nori)']),
]

# Import and Process the Data

In [ ]:
# load the metadata containing file paths
all_metadata = LoadMetadata(file='../metadata.yml', local_data_root='../../data')

# filter epochs for each feeding condition and perform calculations
data = {}
for feeding_condition, data_set_name, channels_to_keep, time_windows_to_keep, epoch_types_to_keep in feeding_condition_parameters:
    data[feeding_condition] = {}
    data[feeding_condition]['data_set_name'] = data_set_name
    data[feeding_condition]['channels_to_keep'] = channels_to_keep
    data[feeding_condition]['time_windows_to_keep'] = time_windows_to_keep
    data[feeding_condition]['epoch_types_to_keep'] = epoch_types_to_keep
    
    blk, annotations_dataframe, epoch_encoder_dataframe, spikes_dataframe = LoadAndPrepareData(all_metadata[data_set_name])
    df = annotations_dataframe.copy()

    # filter by time window
    df = df[np.any(list(map(lambda t: (t[0] <= df['Start (s)']) & (df['End (s)'] <= t[1]), time_windows_to_keep)), axis=0)]
    
    # filter by epoch type
    df = df[np.any(list(map(lambda epoch_type: df['Type'] == epoch_type, epoch_types_to_keep)), axis=0)]

    # calculate interbehavior interval assuming all epochs are from a single contiguous sequence
    df.insert(column='Interval before (s)', loc=3, value=np.nan)
    df.insert(column='Interval after (s)',  loc=4, value=np.nan)
    previous_i = np.nan
    previous_end_time = np.nan
    for i in df.index:
        if not np.isnan(previous_i):
            df.loc[i,          'Interval before (s)'] = df.loc[i, 'Start (s)'] - df.loc[previous_i, 'End (s)']
            df.loc[previous_i, 'Interval after (s)']  = df.loc[i, 'Start (s)'] - df.loc[previous_i, 'End (s)']
        previous_i = i

    # find rectified area under the curve (RAUC) for each channel in each epoch
    for i in df.index:
        behavior_start = df.loc[i, 'Start (s)']
        behavior_end = df.loc[i, 'End (s)']
        behavior_duration = df.loc[i, 'Duration (s)']
        for j in channels_to_keep:
            sig = next((sig for sig in blk.segments[0].analogsignals if sig.name.replace('-L','')==j), None)
            sig = sig.time_slice(behavior_start*pq.s, behavior_end*pq.s)
            rauc = elephant.signal_processing.rauc(sig, baseline='mean').rescale('uV*s')
            df.loc[i, j+' RAUC (uV*s)'] = rauc
            df.loc[i, j+' mean rectified voltage (uV)'] = rauc/(behavior_duration * pq.s)
    
    # add defaults for new columns
    for j in channels_to_keep:
        burst_type = j+' burst'
        df.insert(column=burst_type+' start (s)', loc=len(df.columns), value=np.nan)
        df.insert(column=burst_type+' end (s)', loc=len(df.columns), value=np.nan)
        df.insert(column=burst_type+' RAUC (uV*s)', loc=len(df.columns), value=np.nan)
        df.insert(column=burst_type+' mean rectified voltage (uV)', loc=len(df.columns), value=np.nan)
    df.insert(column='Force start (s)', loc=len(df.columns), value=np.nan)
    df.insert(column='Force end (s)', loc=len(df.columns), value=np.nan)
    df.insert(column='Force RAUC (mN*s)', loc=len(df.columns), value=np.nan)
    df.insert(column='Force mean (mN)', loc=len(df.columns), value=np.nan)
    df.insert(column='Force peak (mN)', loc=len(df.columns), value=np.nan)
    
    # for each behavior
    for i in df.index:
        behavior_start = df.loc[i, 'Start (s)']
        behavior_end = df.loc[i, 'End (s)']
        
        # for each nerve/muscle signal
        for j in channels_to_keep:
            burst_type = j+' burst'
            
            # find a burst epoch on the nerve/muscle contained within this behavior
            df2 = annotations_dataframe.copy()
            df2 = df2[(behavior_start <= df2['Start (s)']) & (df2['End (s)'] <= behavior_end) & (df2['Type'] == burst_type)]
            assert len(df2) < 2, 'More than one epoch found with type \"{}\" for the behavior spanning [{}, {}]'.format(burst_type, behavior_start, behavior_end)
            
            # quantify the burst if it exists
            if len(df2) == 1:
                burst_epoch = df2.iloc[0]
                burst_start = burst_epoch['Start (s)']
                burst_end = burst_epoch['End (s)']
                burst_duration = burst_epoch['Duration (s)']
                
                df.loc[i, burst_type+' start (s)'] = burst_start
                df.loc[i, burst_type+' end (s)'] = burst_end
                
                sig = next((sig for sig in blk.segments[0].analogsignals if sig.name.replace('-L','')==j), None)
                sig = sig.time_slice(behavior_start*pq.s, behavior_end*pq.s)
                rauc = elephant.signal_processing.rauc(sig, baseline='mean', t_start=burst_start*pq.s, t_stop=burst_end*pq.s).rescale('uV*s')
                df.loc[i, burst_type+' RAUC (uV*s)'] = rauc
                df.loc[i, burst_type+' mean rectified voltage (uV)'] = rauc/(burst_duration * pq.s)
            
        # for the force channel
        if epoch_encoder_dataframe is not None:
            
            # find a force epoch that begins within this behavior
            df3 = epoch_encoder_dataframe.copy()
            df3 = df3[(behavior_start <= df3['Start (s)']) & (df3['Start (s)'] <= behavior_end) & (df3['Type'] == 'force')]
            assert len(df3) < 2, 'More than one force epoch begins within the behavior spanning [{}, {}]'.format(behavior_start, behavior_end)
            
            # quantify the force if it exists
            if len(df3) == 1:
                force_epoch = df3.iloc[0]
                force_start = force_epoch['Start (s)']
                force_end = force_epoch['End (s)']
                force_duration = force_epoch['Duration (s)']
                
                df.loc[i, 'Force start (s)'] = force_start
                df.loc[i, 'Force end (s)'] = force_end
                
                sig = next((sig for sig in blk.segments[0].analogsignals if sig.name=='Force'), None)
                sig = sig.time_slice(force_start*pq.s, force_end*pq.s)
                rauc = elephant.signal_processing.rauc(sig).rescale('mN*s')
                df.loc[i, 'Force RAUC (mN*s)'] = rauc
                df.loc[i, 'Force mean (mN)'] = sig.rescale('mN').mean()
                df.loc[i, 'Force peak (mN)'] = sig.rescale('mN').max()
    
    data[feeding_condition]['annotations'] = df

In [ ]:
print('-- Data sets with force blocked out for ANY behavior --')
for feeding_condition in data:
    df = data[feeding_condition]['annotations']
    if np.any(np.isfinite(df['Force start (s)'])):
        print(feeding_condition)
print()
print('-- Data sets with force blocked out for ALL behaviors --')
for feeding_condition in data:
    df = data[feeding_condition]['annotations']
    if np.all(np.isfinite(df['Force start (s)'])):
        print(feeding_condition)

# Plots

In [ ]:
# color map
cm = plt.cm.cool
# cm = plt.cm.brg
# cm = plt.cm.RdBu

sns.set(
#     context = 'poster',
    style = 'ticks',
    font_scale = 1,
    font = 'Palatino Linotype',
)

In [ ]:
duration_range = [0, 25]
interval_range = [-2, 27]
rauc_range = [0, 90]
voltage_mean_range = [0, 10]
force_peak_range = [0, 250]
force_mean_range = [0, 100]
hist_y_range = [0, 25]

markers = ['.', '+', 'x', '1', '4']
colors = ['C0', 'C1', 'C2', 'C3', 'C4']

In [ ]:
# ROWS: ANIMALS, COLS: FOODS
plot_layout = np.array([
    ['JG08 fresh food', 'JG08 regular nori', 'JG08 tape nori', 'JG08 two-ply nori', 'JG08 tubing'],
    [None,              'JG07 regular nori', 'JG07 tape nori', None,                None         ],
    [None,              'JG05 regular nori', 'JG05 tape nori', None,                None         ],
])

n_rows = len(plot_layout)
n_cols = len(plot_layout[0])
for row in plot_layout:
    assert len(row) == n_cols, 'plot_layout needs to be rectangular (not ragged)'

# fig size in inches
figsize = (15,8)

In [ ]:
# # ROWS: FOODS, COLS: ANIMALS
# plot_layout = np.array([
#     ['JG08 fresh food', 'JG08 regular nori', 'JG08 tape nori', 'JG08 two-ply nori', 'JG08 tubing'],
#     [None,              'JG07 regular nori', 'JG07 tape nori', None,                None         ],
#     [None,              'JG05 regular nori', 'JG05 tape nori', None,                None         ],
# ]).T

# n_rows = len(plot_layout)
# n_cols = len(plot_layout[0])
# for row in plot_layout:
#     assert len(row) == n_cols, 'plot_layout needs to be rectangular (not ragged)'

# # fig size in inches
# figsize = (8,12)

In [ ]:
# # ALL ONE ROW
# plot_layout = np.array([
#     ['JG08 fresh food', 'JG08 regular nori', 'JG08 tape nori', 'JG08 two-ply nori', 'JG08 tubing', 'JG07 regular nori', 'JG07 tape nori', 'JG05 regular nori', 'JG05 tape nori'],
# ])

# n_rows = len(plot_layout)
# n_cols = len(plot_layout[0])
# for row in plot_layout:
#     assert len(row) == n_cols, 'plot_layout needs to be rectangular (not ragged)'

# # fig size in inches
# figsize = (30,3)

## Sequence of behavior durations

In [ ]:
plt.figure(1, figsize=figsize)

ylabel = 'Duration (s)'
ylim = duration_range

for i in range(n_rows):
    for j in range(n_cols):
        feeding_condition = plot_layout[i, j]
        if feeding_condition is not None:
            plt.subplot(n_rows, n_cols, i*n_cols+j+1)
            df = data[feeding_condition]['annotations']
            if ylabel in df:
                df[ylabel].plot(marker='.')
                plt.ylim(ylim)
                plt.title(feeding_condition)
                plt.xlabel('Behavior index')
                plt.ylabel(ylabel)
plt.tight_layout()

## Distribution of behavior durations

In [ ]:
# plt.figure(2, figsize=figsize)

# xlabel = 'Duration (s)'
# bins = np.arange(duration_range[0], duration_range[1])

# for i in range(n_rows):
#     for j in range(n_cols):
#         feeding_condition = plot_layout[i, j]
#         if feeding_condition is not None:
#             plt.subplot(n_rows, n_cols, i*n_cols+j+1)
#             df = data[feeding_condition]['annotations']
            
#             n_drop_from_beginning = 0
#             df[xlabel].drop(df.index[:n_drop_from_beginning]).hist(bins=bins)
            
#             n_drop_from_beginning = 5
#             df[xlabel].drop(df.index[:n_drop_from_beginning]).hist(bins=bins)
            
# #             plt.xticks(bins)
#             plt.ylim(hist_y_range)
#             plt.title(feeding_condition)
#             plt.xlabel(xlabel)
#             plt.ylabel('Frequency')
# #             plt.legend(['Full sequence', 'First 5 dropped'])
# plt.tight_layout()

## Sequence of interbehavior intervals

In [ ]:
plt.figure(3, figsize=figsize)

ylabel = 'Interval after (s)'
ylim = interval_range

for i in range(n_rows):
    for j in range(n_cols):
        feeding_condition = plot_layout[i, j]
        if feeding_condition is not None:
            plt.subplot(n_rows, n_cols, i*n_cols+j+1)
            df = data[feeding_condition]['annotations']
            if ylabel in df:
                df[ylabel].plot(marker='.')
                plt.ylim(ylim)
                plt.title(feeding_condition)
                plt.xlabel('Behavior index')
                plt.ylabel(ylabel)
plt.tight_layout()

## Distribution of interbehavior intervals

In [ ]:
# plt.figure(4, figsize=figsize)

# xlabel = 'Interval after (s)'
# bins = np.arange(interval_range[0], interval_range[1])

# for i in range(n_rows):
#     for j in range(n_cols):
#         feeding_condition = plot_layout[i, j]
#         if feeding_condition is not None:
#             plt.subplot(n_rows, n_cols, i*n_cols+j+1)
#             df = data[feeding_condition]['annotations']
            
#             n_drop_from_beginning = 0
#             df[xlabel].drop(df.index[:n_drop_from_beginning]).hist(bins=bins)
            
#             n_drop_from_beginning = 5
#             df[xlabel].drop(df.index[:n_drop_from_beginning]).hist(bins=bins)
            
# #             plt.xticks(bins)
#             plt.ylim(hist_y_range)
#             plt.title(feeding_condition)
#             plt.xlabel(xlabel)
#             plt.ylabel('Frequency')
# #             plt.legend(['Full sequence', 'First 5 dropped'])
# plt.tight_layout()

## Sequences of RAUCs

In [ ]:
# plt.figure(5, figsize=figsize)

# ylabel = 'I2 RAUC (uV*s)'
# ylim = rauc_range

# for i in range(n_rows):
#     for j in range(n_cols):
#         feeding_condition = plot_layout[i, j]
#         if feeding_condition is not None:
#             plt.subplot(n_rows, n_cols, i*n_cols+j+1)
#             df = data[feeding_condition]['annotations']
#             if ylabel in df:
#                 df[ylabel].plot(marker='.')
#                 plt.ylim(ylim)
#                 plt.title(feeding_condition)
#                 plt.xlabel('Behavior index')
#                 plt.ylabel(ylabel)
# plt.tight_layout()

In [ ]:
# plt.figure(6, figsize=figsize)

# ylabel = 'RN RAUC (uV*s)'
# ylim = rauc_range

# for i in range(n_rows):
#     for j in range(n_cols):
#         feeding_condition = plot_layout[i, j]
#         if feeding_condition is not None:
#             plt.subplot(n_rows, n_cols, i*n_cols+j+1)
#             df = data[feeding_condition]['annotations']
#             if ylabel in df:
#                 df[ylabel].plot(marker='.')
#                 plt.ylim(ylim)
#                 plt.title(feeding_condition)
#                 plt.xlabel('Behavior index')
#                 plt.ylabel(ylabel)
# plt.tight_layout()

In [ ]:
# plt.figure(7, figsize=figsize)

# ylabel = 'BN2 RAUC (uV*s)'
# ylim = rauc_range

# for i in range(n_rows):
#     for j in range(n_cols):
#         feeding_condition = plot_layout[i, j]
#         if feeding_condition is not None:
#             plt.subplot(n_rows, n_cols, i*n_cols+j+1)
#             df = data[feeding_condition]['annotations']
#             if ylabel in df:
#                 df[ylabel].plot(marker='.')
#                 plt.ylim(ylim)
#                 plt.title(feeding_condition)
#                 plt.xlabel('Behavior index')
#                 plt.ylabel(ylabel)
# plt.tight_layout()

In [ ]:
# plt.figure(8, figsize=figsize)

# ylabel = 'BN3 RAUC (uV*s)'
# ylim = rauc_range

# for i in range(n_rows):
#     for j in range(n_cols):
#         feeding_condition = plot_layout[i, j]
#         if feeding_condition is not None:
#             plt.subplot(n_rows, n_cols, i*n_cols+j+1)
#             df = data[feeding_condition]['annotations']
#             if ylabel in df:
#                 df[ylabel].plot(marker='.')
#                 plt.ylim(ylim)
#                 plt.title(feeding_condition)
#                 plt.xlabel('Behavior index')
#                 plt.ylabel(ylabel)
# plt.tight_layout()

In [ ]:
plt.figure(9, figsize=figsize)

ylabels = [
    'Duration (s)',
    'I2 RAUC (uV*s)',
    'RN RAUC (uV*s)',
    'BN2 RAUC (uV*s)',
    'BN3 RAUC (uV*s)'
]
ylim = [np.min([duration_range, rauc_range]), np.max([duration_range, rauc_range])]

for i in range(n_rows):
    for j in range(n_cols):
        feeding_condition = plot_layout[i, j]
        if feeding_condition is not None:
            plt.subplot(n_rows, n_cols, i*n_cols+j+1)
            df = data[feeding_condition]['annotations']
            for ylabel in ylabels:
                if ylabel in df:
                    df[ylabel].plot(marker='.')
                    plt.ylim(ylim)
                    plt.title(feeding_condition)
                    plt.xlabel('Behavior index')
                    plt.ylabel('Duration or RAUC')
#             plt.legend()
plt.tight_layout()

## Sequences of mean rectified voltages

In [ ]:
plt.figure(10, figsize=figsize)

ylabels = [
    'Duration (s)',
    'I2 mean rectified voltage (uV)',
    'RN mean rectified voltage (uV)',
    'BN2 mean rectified voltage (uV)',
    'BN3 mean rectified voltage (uV)',
]
ylim = [np.min([duration_range, voltage_mean_range]), np.max([duration_range, voltage_mean_range])]

for i in range(n_rows):
    for j in range(n_cols):
        feeding_condition = plot_layout[i, j]
        if feeding_condition is not None:
            plt.subplot(n_rows, n_cols, i*n_cols+j+1)
            df = data[feeding_condition]['annotations']
            for ylabel in ylabels:
                if ylabel in df:
                    df[ylabel].plot(marker='.')
                    plt.ylim(ylim)
                    plt.title(feeding_condition)
                    plt.xlabel('Behavior index')
                    plt.ylabel('Duration or mean rectified voltage')
#             plt.legend()
plt.tight_layout()

## Scatter plots of behavior durations and mean rectified voltages

In [ ]:
plt.figure(11, figsize=(5, 12))

# feeding_conditions = ['JG08 regular nori', 'JG08 tape nori', 'JG08 two-ply nori', 'JG08 fresh food', 'JG08 tubing']
feeding_conditions = ['JG08 regular nori', 'JG08 tape nori', 'JG08 two-ply nori', None, None]
# feeding_conditions = [None, None, None, 'JG08 fresh food', None]

xlabel = 'Duration (s)'
ylabels = ['I2 mean rectified voltage (uV)', 'RN mean rectified voltage (uV)', 'BN2 mean rectified voltage (uV)', 'BN3 mean rectified voltage (uV)']

xlim = duration_range
ylim = voltage_mean_range

legend = []
for i, ylabel in enumerate(ylabels):
    plt.subplot(len(ylabels), 1, i+1)
    for j, feeding_condition in enumerate(feeding_conditions):
        if feeding_condition is not None:
            legend.append(feeding_condition)
            df = data[feeding_condition]['annotations']
            x = df[xlabel]
            y = df[ylabel]
            plt.scatter(x, y, marker=markers[j], c=colors[j])
    plt.xlabel(xlabel)
    plt.xlim(xlim)
    plt.ylabel(ylabel)
    plt.ylim(ylim)
    plt.legend(legend)
plt.tight_layout()

In [ ]:
plt.figure(12, figsize=(5, 5))
ax = plt.gca(projection='3d')

# feeding_conditions = ['JG08 regular nori', 'JG08 tape nori', 'JG08 two-ply nori', 'JG08 fresh food', 'JG08 tubing']
feeding_conditions = ['JG08 regular nori', 'JG08 tape nori', 'JG08 two-ply nori', None, None]
# feeding_conditions = [None, None, None, 'JG08 fresh food', None]

xlabel = 'Duration (s)'
ylabel = 'BN2 mean rectified voltage (uV)'
zlabel = 'BN3 mean rectified voltage (uV)'

xlim = duration_range
ylim = voltage_mean_range
zlim = voltage_mean_range

legend = []
for j, feeding_condition in enumerate(feeding_conditions):
    if feeding_condition is not None:
        legend.append(feeding_condition)
        df = data[feeding_condition]['annotations']
        x = df[xlabel]
        y = df[ylabel]
        z = df[zlabel]
        ax.scatter(x, y, z, marker=markers[j], c=colors[j])#, depthshade=False)
ax.set_xlabel(xlabel)
ax.set_xlim(xlim)
ax.set_ylabel(ylabel)
ax.set_ylim(ylim)
ax.set_zlabel(zlabel)
ax.set_zlim(zlim)
ax.legend(legend)
plt.tight_layout()

In [ ]:
plt.figure(13, figsize=(5, 12))

feeding_conditions = ['JG07 regular nori', 'JG07 tape nori']

xlabel = 'Duration (s)'
ylabels = ['I2 mean rectified voltage (uV)', 'RN mean rectified voltage (uV)', 'BN2 mean rectified voltage (uV)', 'BN3 mean rectified voltage (uV)']

xlim = duration_range
ylim = voltage_mean_range

legend = []
for i, ylabel in enumerate(ylabels):
    if ylabel is not None:
        plt.subplot(len(ylabels), 1, i+1)
        for j, feeding_condition in enumerate(feeding_conditions):
            if feeding_condition is not None:
                legend.append(feeding_condition)
                df = data[feeding_condition]['annotations']
                x = df[xlabel]
                y = df[ylabel]
                plt.scatter(x, y, marker=markers[j], c=colors[j])
        plt.xlabel(xlabel)
        plt.xlim(xlim)
        plt.ylabel(ylabel)
        plt.ylim(ylim)
        plt.legend(legend)
plt.tight_layout()

In [ ]:
plt.figure(14, figsize=(5, 5))
ax = plt.gca(projection='3d')

feeding_conditions = ['JG07 regular nori', 'JG07 tape nori']

xlabel = 'Duration (s)'
ylabel = 'BN2 mean rectified voltage (uV)'
zlabel = 'BN3 mean rectified voltage (uV)'

xlim = duration_range
ylim = voltage_mean_range
zlim = voltage_mean_range

legend = []
for j, feeding_condition in enumerate(feeding_conditions):
    if feeding_condition is not None:
        legend.append(feeding_condition)
        df = data[feeding_condition]['annotations']
        x = df[xlabel]
        y = df[ylabel]
        z = df[zlabel]
        ax.scatter(x, y, z, marker=markers[j], c=colors[j])#, depthshade=False)
ax.set_xlabel(xlabel)
ax.set_xlim(xlim)
ax.set_ylabel(ylabel)
ax.set_ylim(ylim)
ax.set_zlabel(zlabel)
ax.set_zlim(zlim)
ax.legend(legend)
plt.tight_layout()

In [ ]:
plt.figure(15, figsize=(5, 12))

feeding_conditions = ['JG05 regular nori', 'JG05 tape nori']

xlabel = 'Duration (s)'
ylabels = ['I2 mean rectified voltage (uV)', None, 'BN2 mean rectified voltage (uV)', 'BN3 mean rectified voltage (uV)']

xlim = duration_range
ylim = voltage_mean_range

legend = []
for i, ylabel in enumerate(ylabels):
    if ylabel is not None:
        plt.subplot(len(ylabels), 1, i+1)
        for j, feeding_condition in enumerate(feeding_conditions):
            if feeding_condition is not None:
                legend.append(feeding_condition)
                df = data[feeding_condition]['annotations']
                x = df[xlabel]
                y = df[ylabel]
                plt.scatter(x, y, marker=markers[j], c=colors[j])
        plt.xlabel(xlabel)
        plt.xlim(xlim)
        plt.ylabel(ylabel)
        plt.ylim(ylim)
        plt.legend(legend)
plt.tight_layout()

In [ ]:
plt.figure(16, figsize=(5, 5))
ax = plt.gca(projection='3d')

feeding_conditions = ['JG05 regular nori', 'JG05 tape nori']

xlabel = 'Duration (s)'
ylabel = 'BN2 mean rectified voltage (uV)'
zlabel = 'BN3 mean rectified voltage (uV)'

xlim = duration_range
ylim = voltage_mean_range
zlim = voltage_mean_range

legend = []
for j, feeding_condition in enumerate(feeding_conditions):
    if feeding_condition is not None:
        legend.append(feeding_condition)
        df = data[feeding_condition]['annotations']
        x = df[xlabel]
        y = df[ylabel]
        z = df[zlabel]
        ax.scatter(x, y, z, marker=markers[j], c=colors[j])#, depthshade=False)
ax.set_xlabel(xlabel)
ax.set_xlim(xlim)
ax.set_ylabel(ylabel)
ax.set_ylim(ylim)
ax.set_zlabel(zlabel)
ax.set_zlim(zlim)
ax.legend(legend)
plt.tight_layout()

## Force plots

In [ ]:
plt.figure(17, figsize=(5, 5))
ax = plt.gca(projection='3d')

feeding_conditions = ['JG07 tape nori', 'JG08 tape nori']

xlabel = 'Duration (s)'
ylabel = 'Force peak (mN)'
zlabel = 'Force mean (mN)'

xlim = duration_range
ylim = force_peak_range
zlim = force_mean_range

legend = []
for j, feeding_condition in enumerate(feeding_conditions):
    if feeding_condition is not None:
        legend.append(feeding_condition)
        df = data[feeding_condition]['annotations']
        x = df[xlabel]
        y = df[ylabel]
        z = df[zlabel]
        ax.scatter(x, y, z, marker=markers[j], c=colors[j])#, depthshade=False)
ax.set_xlabel(xlabel)
ax.set_xlim(xlim)
ax.set_ylabel(ylabel)
ax.set_ylim(ylim)
ax.set_zlabel(zlabel)
ax.set_zlim(zlim)
ax.legend(legend)
plt.tight_layout()

In [ ]:
plt.figure(18, figsize=(5, 5))
ax = plt.gca(projection='3d')

feeding_conditions = ['JG07 tape nori', 'JG08 tape nori']

xlabel = 'Duration (s)'
ylabel = 'BN2 mean rectified voltage (uV)'
zlabel = 'Force mean (mN)'

xlim = duration_range
ylim = voltage_mean_range
zlim = force_mean_range

legend = []
for j, feeding_condition in enumerate(feeding_conditions):
    if feeding_condition is not None:
        legend.append(feeding_condition)
        df = data[feeding_condition]['annotations']
        x = df[xlabel]
        y = df[ylabel]
        z = df[zlabel]
        ax.scatter(x, y, z, marker=markers[j], c=colors[j])#, depthshade=False)
ax.set_xlabel(xlabel)
ax.set_xlim(xlim)
ax.set_ylabel(ylabel)
ax.set_ylim(ylim)
ax.set_zlabel(zlabel)
ax.set_zlim(zlim)
ax.legend(legend)
plt.tight_layout()

In [ ]:
plt.figure(19, figsize=(5, 5))
ax = plt.gca(projection='3d')

feeding_conditions = ['JG07 tape nori', 'JG08 tape nori']

xlabel = 'Duration (s)'
ylabel = 'BN2 mean rectified voltage (uV)'
zlabel = 'Force peak (mN)'

xlim = duration_range
ylim = voltage_mean_range
zlim = force_peak_range

legend = []
for j, feeding_condition in enumerate(feeding_conditions):
    if feeding_condition is not None:
        legend.append(feeding_condition)
        df = data[feeding_condition]['annotations']
        x = df[xlabel]
        y = df[ylabel]
        z = df[zlabel]
        ax.scatter(x, y, z, marker=markers[j], c=colors[j])#, depthshade=False)
ax.set_xlabel(xlabel)
ax.set_xlim(xlim)
ax.set_ylabel(ylabel)
ax.set_ylim(ylim)
ax.set_zlabel(zlabel)
ax.set_zlim(zlim)
ax.legend(legend)
plt.tight_layout()